In [1]:
# Import needed packages
import os
import pandas as pd
import numpy as np

In [2]:
# Get Working Directory
os.getcwd()

'c:\\Users\\bfran\\Ironhack\\Week 3\\Project\\Ironhack-PRY-BRZ-MGA'

In [3]:
# Load all dataframes for merging

# Save tariff dataframes
brz_app_tariffs = pd.read_csv('brz_app_tariffs.csv')
brz_bnd_tariffs = pd.read_csv('brz_bnd_tariffs.csv')

# Save ITIC dataframe
brz_itic = pd.read_csv('brz_ave_itic.csv')

# Save import/export dataframes
pry_exports = pd.read_csv('cleaned py exports to br.csv')
brz_imports = pd.read_csv('cleaned BRZ Imports from PRY.csv')

In [4]:
# Subsets tariff data to only include most recent year
brz_app_tariffs = brz_app_tariffs[brz_app_tariffs['year']==2023]

In [5]:
# Saves subsets with only those columns that have data we want for our combined dataframe
s_brz_app_tariffs = brz_app_tariffs.loc[:,['hs_code','applied_tariff']]
s_brz_bnd_tariffs = brz_bnd_tariffs.loc[:,['hs_code','bound_tariff']]
s_brz_itic = brz_itic.loc[:,['hs_4_code','ave_itic_rate']]

In [6]:
# Merges export and import data into a single dataframe for comparison
mx_compare = pd.merge(pry_exports,brz_imports, on="hs_code", how="outer")

In [7]:
# Renames columns to be more descriptive, facilitating understanding of dataframe
# Key:
    # pry = Paraguay
    # brz = Brazil
    # x = export
    # m = import
    # q = quantity
    # value = value before tariffs and before transportation costs
mx_compare.rename(columns = {'quantity_x':'pry_x_q',
                             'net_weight_x':'pry_x_net_wgt',
                             'unit_type_x':'pry_x_unit',
                             'quantity_y':'brz_m_q',
                             'net_weight_y':'brz_m_net_wgt',
                             'unit_type_y':'brz_m_unit',
                             'fob_value_x':'pry_x_value',
                             'fob_value_y':'brz_m_value'},
                             inplace = True)

In [8]:
# Saves subset of import/export comparison dataframe w/ only hs codes, quantities, units, weights, and values
# Organizes columns to be in a more logical order
mx_compare = mx_compare.loc[:,['hs_code','pry_x_q','pry_x_unit','brz_m_q','brz_m_unit','pry_x_net_wgt','brz_m_net_wgt','pry_x_value','brz_m_value']]

In [9]:
# Merges into condensed import/export comparison the subsets of tariff data on the hs_code
mx_compare = pd.merge(mx_compare, s_brz_app_tariffs, on='hs_code', how='inner')
mx_compare = pd.merge(mx_compare, s_brz_bnd_tariffs, on='hs_code', how='inner')

# Displays updated dataframe
mx_compare

,hs_code,pry_x_q,pry_x_unit,brz_m_q,brz_m_unit,pry_x_net_wgt,brz_m_net_wgt,pry_x_value,brz_m_value,applied_tariff,bound_tariff
0,20130,21523200.0,kg,20859060.0,kg,21523200.0,2.085906e+07,1.249529e+08,120517874.0,9.600000,55.000000
1,20220,52442.3,kg,52443.0,kg,52442.3,5.244300e+04,1.404194e+05,140643.0,8.000000,55.000000
2,20230,7425400.0,kg,7144609.0,kg,7425400.0,7.144609e+06,3.770901e+07,36218864.0,9.600000,55.000000
3,20622,2883500.0,kg,2692900.0,kg,2883500.0,2.692900e+06,3.287412e+06,3053319.0,8.000000,35.000000
4,20629,1037276.0,kg,978823.0,kg,1037276.0,9.788230e+05,2.882343e+06,2749729.0,8.000000,35.000000
...,...,...,...,...,...,...,...,...,...,...,...
791,851230,NaN,NaN,2.0,u,NaN,1.920000e-01,NaN,4.0,18.000000,35.000000
792,870810,NaN,NaN,15.0,kg,NaN,1.500000e+01,NaN,10.0,18.000000,25.000000
793,870894,NaN,NaN,2.0,kg,NaN,2.000000e+00,NaN,25.0,15.085714,28.333333
794,902610,NaN,NaN,102.0,u,NaN,8.480000e+00,NaN,1278.0,11.800000,21.500000


In [10]:
# Converts hs_code column to a string
mx_compare['hs_code'] = mx_compare['hs_code'].astype(str)
s_brz_itic['hs_4_code'] = s_brz_itic['hs_4_code'].astype(str)

In [11]:
# Adds a 0 to the start of every HS code that is lacking the leading 0.
char_to_add = '0'
mx_compare['hs_code'] = mx_compare['hs_code'].apply(lambda x : char_to_add + x if len(x) == 5 else x)
s_brz_itic['hs_4_code'] = s_brz_itic['hs_4_code'].apply(lambda x : char_to_add + x if len(x) == 3 else x)

In [12]:
# Create a mapping from the first four digits to category
mapping = pd.Series(s_brz_itic['ave_itic_rate'].values, index=s_brz_itic['hs_4_code']).to_dict()

In [13]:
# Define itic_mapping function to populate itic data in main...
# ... dataframe w/ proper corresponding itic rates.
def itic_mapping(row, mapping):
    first_four = row['hs_code'][:4]
    return mapping.get(first_four,pd.NA)

In [14]:
# Maps itic rates to proper rows based on hs code
mx_compare['itic_rate'] = mx_compare.apply(itic_mapping, mapping=mapping, axis=1)

In [15]:
# Display column names
mx_compare.columns

Index(['hs_code', 'pry_x_q', 'pry_x_unit', 'brz_m_q', 'brz_m_unit',
       'pry_x_net_wgt', 'brz_m_net_wgt', 'pry_x_value', 'brz_m_value',
       'applied_tariff', 'bound_tariff', 'itic_rate'],
      dtype='object')

In [16]:
# Reorders columns to make more sense
mx_compare = mx_compare[['hs_code', 'pry_x_q', 'pry_x_unit', 'brz_m_q', 'brz_m_unit','pry_x_net_wgt', 'brz_m_net_wgt', 'pry_x_value','itic_rate', 'applied_tariff', 'bound_tariff','brz_m_value']]

In [17]:
# Drops bound tariff column because applied tariff...
# ... has no nulls. Would've kept and used otherwise.
mx_compare.drop(columns='bound_tariff',inplace = True)

# Ignore warning because we do want to drop column.

C:\Users\bfran\AppData\Local\Temp\ipykernel_14308\3137133380.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mx_compare.drop(columns='bound_tariff',inplace = True)


In [18]:
# Fills null values in most numeric columns with 0
# Fills null values in non-numeric columns with NR, standing for Not Reported
# Fills null vaues in itic_rate column with the average of that column. 
# We chose to fill w/ average b/c if a country's trade patterns tend towards...
# ... more or less expensive shipping, we want that to be reflected in the...
# ... itic values we have to fill. 
mx_compare['pry_x_q'].fillna(0, inplace=True)
mx_compare['pry_x_unit'].fillna('NR', inplace=True)
mx_compare['brz_m_q'].fillna(0, inplace=True)
mx_compare['brz_m_unit'].fillna("NR", inplace=True)
mx_compare['pry_x_net_wgt'].fillna(0, inplace=True)
mx_compare['brz_m_net_wgt'].fillna(0, inplace=True)
mx_compare['pry_x_value'].fillna(0, inplace=True)
mx_compare['itic_rate'].fillna(mx_compare['itic_rate'].mean(), inplace=True)
mx_compare['brz_m_value'].fillna(0, inplace=True)

# Ignore warnings because we want what it's warning us about to happen. 

C:\Users\bfran\AppData\Local\Temp\ipykernel_14308\3040777839.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mx_compare['pry_x_q'].fillna(0, inplace=True)
C:\Users\bfran\AppData\Local\Temp\ipykernel_14308\3040777839.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mx_compare['pry_x_unit'].fillna('NR', inplace=True)
C:\Users\bfran\AppData\Local\Temp\ipykernel_14308\3040777839.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mx

In [19]:
# Defines new columns to create and populate based on arithmetic operations...
# ... run using other columns. 

# Multiplies reported export value by itic rate to account for shipping cost.
mx_compare['adj_x_value'] = mx_compare['pry_x_value'] * mx_compare['itic_rate']

# Subtracts reported import value from export value to identify value gap. 
mx_compare['trade_gap'] = mx_compare['adj_x_value'] - mx_compare['brz_m_value']

# Multiples trade gap by tariff rate to estimate amount of tariff lost due to gap. 
# This would identify which HS codes need the most attention of gov't action...
# ... because bigger tariff losses necessitate greater attention.
mx_compare['est_tax_loss'] = mx_compare['trade_gap'] * (mx_compare['applied_tariff']/100)

# Subtracts reported import weight from import weight to identify weight gap.
mx_compare['wgt_gap'] = mx_compare['pry_x_net_wgt'] - mx_compare['brz_m_net_wgt']

# The closer any of the following ratios are to 1, the more consistent...
# ... reporting was between Brazil and Paraguay. The farther from 1...
# ... the bigger a discrepancy there is. 

# Divides value of imports by value of exports to establish a value ratio.
mx_compare['value_ratio'] = mx_compare['brz_m_value'] / mx_compare['adj_x_value']

# Divides weight of imports by weight of exports to establish a weight ratio.
mx_compare['wgt_ratio'] = mx_compare['brz_m_net_wgt'] / mx_compare['pry_x_net_wgt']

# Divides value ratio by weight ratio. 
mx_compare['density_ratio'] = mx_compare['value_ratio'] / mx_compare['wgt_ratio']

In [20]:
# Displays column names and order
mx_compare.columns

Index(['hs_code', 'pry_x_q', 'pry_x_unit', 'brz_m_q', 'brz_m_unit',
       'pry_x_net_wgt', 'brz_m_net_wgt', 'pry_x_value', 'itic_rate',
       'applied_tariff', 'brz_m_value', 'adj_x_value', 'trade_gap',
       'est_tax_loss', 'wgt_gap', 'value_ratio', 'wgt_ratio', 'density_ratio'],
      dtype='object')

In [21]:
# Reorders columns again for greater understanding and legibility
mx_compare = mx_compare[['hs_code', 'pry_x_q', 'pry_x_unit', 'brz_m_q', 
                         'brz_m_unit','pry_x_net_wgt', 'brz_m_net_wgt', 
                         'pry_x_value', 'itic_rate','applied_tariff',  
                         'adj_x_value', 'brz_m_value','trade_gap',
                         'est_tax_loss', 'wgt_gap', 'value_ratio', 
                         'wgt_ratio', 'density_ratio']]

# Displays new dataframe
mx_compare

,hs_code,pry_x_q,pry_x_unit,brz_m_q,brz_m_unit,pry_x_net_wgt,brz_m_net_wgt,pry_x_value,itic_rate,applied_tariff,adj_x_value,brz_m_value,trade_gap,est_tax_loss,wgt_gap,value_ratio,wgt_ratio,density_ratio
0,020130,21523200.0,kg,20859060.0,kg,21523200.0,2.085906e+07,1.249529e+08,1.013000,9.600000,1.265773e+08,120517874.0,6.059425e+06,581704.772286,664140.000,0.952129,0.969143,0.982444
1,020220,52442.3,kg,52443.0,kg,52442.3,5.244300e+04,1.404194e+05,1.009333,8.000000,1.417300e+05,140643.0,1.087011e+03,86.960908,-0.700,0.992330,1.000013,0.992317
2,020230,7425400.0,kg,7144609.0,kg,7425400.0,7.144609e+06,3.770901e+07,1.009333,9.600000,3.806096e+07,36218864.0,1.842098e+06,176841.369384,280791.000,0.951601,0.962185,0.989000
3,020622,2883500.0,kg,2692900.0,kg,2883500.0,2.692900e+06,3.287412e+06,1.033500,8.000000,3.397540e+06,3053319.0,3.442215e+05,27537.719042,190600.000,0.898685,0.933900,0.962293
4,020629,1037276.0,kg,978823.0,kg,1037276.0,9.788230e+05,2.882343e+06,1.033500,8.000000,2.978902e+06,2749729.0,2.291729e+05,18333.833966,58453.000,0.923068,0.943648,0.978191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,851230,0.0,NR,2.0,u,0.0,1.920000e-01,0.000000e+00,1.031167,18.000000,0.000000e+00,4.0,-4.000000e+00,-0.720000,-0.192,inf,inf,NaN
792,870810,0.0,NR,15.0,kg,0.0,1.500000e+01,0.000000e+00,1.007667,18.000000,0.000000e+00,10.0,-1.000000e+01,-1.800000,-15.000,inf,inf,NaN
793,870894,0.0,NR,2.0,kg,0.0,2.000000e+00,0.000000e+00,1.007667,15.085714,0.000000e+00,25.0,-2.500000e+01,-3.771429,-2.000,inf,inf,NaN
794,902610,0.0,NR,102.0,u,0.0,8.480000e+00,0.000000e+00,1.014000,11.800000,0.000000e+00,1278.0,-1.278000e+03,-150.804000,-8.480,inf,inf,NaN


In [22]:
# Checks for duplicate rows, returns the sum. 
mx_compare.duplicated().sum()

0

In [23]:
# Checks null values in each column
mx_compare.isnull().sum()

hs_code             0
pry_x_q             0
pry_x_unit          0
brz_m_q             0
brz_m_unit          0
pry_x_net_wgt       0
brz_m_net_wgt       0
pry_x_value         0
itic_rate           0
applied_tariff      0
adj_x_value         0
brz_m_value         0
trade_gap           0
est_tax_loss        0
wgt_gap             0
value_ratio         0
wgt_ratio           1
density_ratio     372
dtype: int64

In [24]:
# Creates and displays null weight column to investigate null value. 
null_wgt = mx_compare[mx_compare['wgt_ratio'].isnull() == True]
null_wgt

# Shows that the null is related to the fact that Paraguay exported...
# ... electricity to Brazil, and electricity has no weight. When...
# ... no weights were reported, it caused a dividing by zero error. 

,hs_code,pry_x_q,pry_x_unit,brz_m_q,brz_m_unit,pry_x_net_wgt,brz_m_net_wgt,pry_x_value,itic_rate,applied_tariff,adj_x_value,brz_m_value,trade_gap,est_tax_loss,wgt_gap,value_ratio,wgt_ratio,density_ratio
79,271600,9833311.262,1000 kWh,5674389.713,1000 kWh,0.0,0.0,1.059499e+09,1.069333,0.0,1.132957e+09,1.058352e+09,7.460527e+07,0.0,0.0,0.93415,NaN,NaN


In [25]:
# Defines columns to round
numeric_columns = ['pry_x_q', 'brz_m_q',
       'pry_x_net_wgt', 'brz_m_net_wgt', 'pry_x_value', 'itic_rate',
       'applied_tariff', 'adj_x_value', 'brz_m_value', 'trade_gap',
       'est_tax_loss', 'wgt_gap', 'value_ratio', 'wgt_ratio', 'density_ratio']

# Rounds defined columns to improve legibility
mx_compare[numeric_columns] = mx_compare[numeric_columns].round(2)

In [26]:
# Replace 0.0s and infs in value_ratio and wgt_ratio columns
replacement_mapping = {0.0: 'NMR'}
mx_compare = mx_compare.replace({'value_ratio': replacement_mapping, 'wgt_ratio': replacement_mapping})

inf_columns = ['value_ratio','wgt_ratio']
mx_compare[inf_columns] = mx_compare[inf_columns].replace([np.inf,-np.inf],'NXR')

# Fixes null value in wgt ratio - product was electricity, which has no weight, causing NaN erro
mx_compare['wgt_ratio'].fillna('kWh',inplace=True)

In [27]:
# Set pandas to display numbers instead of scientific notation
pd.options.display.float_format = '{:.2f}'.format

In [28]:
# Displays final cleaned mx_compare dataframe
mx_compare

,hs_code,pry_x_q,pry_x_unit,brz_m_q,brz_m_unit,pry_x_net_wgt,brz_m_net_wgt,pry_x_value,itic_rate,applied_tariff,adj_x_value,brz_m_value,trade_gap,est_tax_loss,wgt_gap,value_ratio,wgt_ratio,density_ratio
0,020130,21523200.00,kg,20859060.00,kg,21523200.00,20859060.00,124952910.87,1.01,9.60,126577298.71,120517874.00,6059424.71,581704.77,664140.00,0.95,0.97,0.98
1,020220,52442.30,kg,52443.00,kg,52442.30,52443.00,140419.43,1.01,8.00,141730.01,140643.00,1087.01,86.96,-0.70,0.99,1.00,0.99
2,020230,7425400.00,kg,7144609.00,kg,7425400.00,7144609.00,37709010.83,1.01,9.60,38060961.60,36218864.00,1842097.60,176841.37,280791.00,0.95,0.96,0.99
3,020622,2883500.00,kg,2692900.00,kg,2883500.00,2692900.00,3287412.18,1.03,8.00,3397540.49,3053319.00,344221.49,27537.72,190600.00,0.90,0.93,0.96
4,020629,1037276.00,kg,978823.00,kg,1037276.00,978823.00,2882343.42,1.03,8.00,2978901.92,2749729.00,229172.92,18333.83,58453.00,0.92,0.94,0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,851230,0.00,NR,2.00,u,0.00,0.19,0.00,1.03,18.00,0.00,4.00,-4.00,-0.72,-0.19,NXR,NXR,NaN
792,870810,0.00,NR,15.00,kg,0.00,15.00,0.00,1.01,18.00,0.00,10.00,-10.00,-1.80,-15.00,NXR,NXR,NaN
793,870894,0.00,NR,2.00,kg,0.00,2.00,0.00,1.01,15.09,0.00,25.00,-25.00,-3.77,-2.00,NXR,NXR,NaN
794,902610,0.00,NR,102.00,u,0.00,8.48,0.00,1.01,11.80,0.00,1278.00,-1278.00,-150.80,-8.48,NXR,NXR,NaN


In [30]:
# Saves cleaned dataframe as a new .csv file for future use. 
mx_compare.to_csv('mx_compare.csv',index=False)